In [ ]:
1+2

In [ ]:
# setup GPU memory limit

# For 1070
g=5.0
# For 2080 Ti
# g=9.0

ENV["JULIA_CUDA_MEMORY_LIMIT"] = convert(Int, round(g * 1024 * 1024 * 1024))

In [ ]:
include("main.jl")

In [ ]:
main_CNN_sep
main_EQ_sep

In [ ]:
main_CNN_sep()

In [ ]:
main_EQ_sep()

In [ ]:
main_FC()

In [ ]:
main_CNN_ensemble()

In [ ]:
main_EQ_ensemble()

In [ ]:
main_EQ_cov()

# New version of main

- refactor the code of main1
- use seeding
- pre-gen graphs and splitting
- use cloud GPU for training

- [ ] I don't need correlation experiments anymore

In [ ]:
include("main2.jl")

In [ ]:
main2()